In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
path = Path("/sc/arion/projects/DiseaseGeneCell/Huang_lab_project/wangcDrugRepoProject/BindDBdata/All_BindingDB/ESM2_BindingDB_random")
df = pd.read_parquet(path / "train.parquet", engine='fastparquet')
df.head()

In [ ]:
path = "/sc/arion/projects/DiseaseGeneCell/Huang_lab_project/drug_discovery/output/models/Random_Forest_ESMv1_BindingDB_random_predictions.csv"
pred_df = pd.read_csv(Path(path))
pred_df.rename(columns={"drug_smile":"Drug", "affinity": "affinity_pred"}, inplace=True)
pred_df.head()